In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.nn import BCELoss, CrossEntropyLoss
from extractor import addImagePath, textExtraction, imageExtraction, textExtractReverse

# 1. Check the max length of the text data


In [2]:
subUrlList = ['wendys','mcdonalds', 'mcdonalds_switzerland','mcdonaldscanada','sonicdrivein']
max_length = 0
sum_length = 0
for subUrl in subUrlList:
    dirPath = '../Data/Instagram/Filter_' + subUrl + '.csv'
    data = pd.read_csv(dirPath)
    
    for i in range(len(data['caption'])):
        sum_length += len(str(data['caption'][i]).split())
        if len(str(data['caption'][i]).split()) > max_length:
            max_length = len(str(data['caption'][i]).split())
            word = data['caption'][i]
print(max_length)
print(sum_length/len(data['caption']))

373
31.870794078061913


In [193]:
max_length = 0
word = ''
dirPath = '../Data/Oxford_HIC/oxford_hic_data.csv'
data = pd.read_csv(dirPath)
max_length = 0
sum_length = 0
index =0 
counter  = 5
# find the max word count of the text data['caption']
for i in range(len(data['caption'])):
    sum_length += len(str(data['caption'][i]).split())
    if len(str(data['caption'][i]).split()) > max_length:
        max_length = len(str(data['caption'][i]).split())
        word = data['caption'][i]
        index = i       
        #
print(max_length, i)
print(sum_length/len(data['caption']))
data.shape

C:\Users\USER\AppData\Local\Temp\ipykernel_5988\3251324240.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(dirPath)


9729 3657846
10.514508397972905


(3657847, 3)

In [195]:
sum_length

38460463

# 2. Check the max word count of the text data

In [260]:
subUrlList = ['wendys','mcdonalds', 'mcdonalds_switzerland','mcdonaldscanada','sonicdrivein']
wordList = []
total = 0
for subUrl in subUrlList:
    dirPath = '../Data/Instagram/Filter_' + subUrl + '.csv'
    data = pd.read_csv(dirPath)
    
    for i in range(len(data['caption'])):
        for word in str(data['caption'][i]).split():
            if word not in wordList:
                wordList.append(word)
                total += 1
                
print(len(wordList))        
print(total)

15056
15056


In [261]:
wordList = []
total = 0
dirPath = '../Data/Oxford_HIC/oxford_hic_data.csv'
data = pd.read_csv(dirPath)

for i in range(len(data['caption'])):
    for word in str(data['caption'][i]).split():
        if word not in wordList:
            wordList.append(word)
            total += 1

print(len(wordList))        
print(total)

C:\Users\USER\AppData\Local\Temp\ipykernel_5988\496614120.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(dirPath)


KeyboardInterrupt: 

# 3. Load the data and split the data

In [2]:
# if args.img - dir == 'Oxford_HIC':
#     dirPath = '../Data/Oxford_HIC/oxford_hic_data.csv'
#     imgPath = '../Data/Oxford_HIC/oxford_img/'
# else:
dirPath = '../Data/Instagram/Filter_' + 'wendys' + '.csv'
imgPath = '../Data/Instagram/' + 'wendys' + '_img/'
# load data
data = pd.read_csv(dirPath)
data = addImagePath(data, imgPath)
# split data
train, test = train_test_split(data, test_size=0.2, random_state=42)

train_text = textExtraction(train['caption'].tolist())
train_image = imageExtraction(train['image_id'])
train_funny_score = torch.tensor(train['funny_score'].to_numpy())
test_text = textExtraction(test['caption'])
test_image = imageExtraction(test['image_id'])
test_funny_score = torch.tensor(test['funny_score'].to_numpy())

C:\Users\USER\PycharmProjects\Python3.11.9\pythonProject\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
100%|██████████| 74/74 [00:01<00:00, 37.90it/s]


In [3]:
train_text.shape, train_image.shape, train_funny_score.shape

(torch.Size([293, 64, 768]), torch.Size([293, 64, 768]), torch.Size([293]))

In [4]:
batch_size = 32
train_dataset = torch.utils.data.TensorDataset(train_text, train_image, train_funny_score)
test_dataset = torch.utils.data.TensorDataset(test_text, test_image, test_funny_score)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 3. LLM Test

In [5]:
### 不確定是否為官方的 Gemini ############################################################################
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("describeai/gemini")
gemini = AutoModelForSeq2SeqLM.from_pretrained("describeai/gemini")
#######################################################################################################
gemini

C:\Users\USER\PycharmProjects\Python3.11.9\pythonProject\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (d

In [13]:
### 官方的Gemma #########################################################################################
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b")
gemma = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b", device_map="auto",  torch_dtype=torch.bfloat16)
########################################################################################################

C:\Users\USER\PycharmProjects\Python3.11.9\pythonProject\.venv\Lib\site-packages\accelerate\utils\modeling.py:1405: UserWarning: Current model requires 4608 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gemma.to(device)
vocab_size = 256128  # 词汇表大小
embedding_dim = 768  # 嵌入维度，与你的图像嵌入维度相同
text_embedding = nn.Embedding(vocab_size, embedding_dim).to(device)

words = "👻🔥😂😁👍🤦‍♀️🤦‍♂️🤷‍♀️🤷‍♂️✌🤞😉😎🎶😢💖🎉🌹💋👏🐱‍💻🐱‍🐉🐱‍👓✔👀😃✨😆🤔🤢🎁🫢 ha ha"
tokens = tokenizer(words, truncation=True, padding= 'max_length', max_length=100, return_tensors="pt")
output = text_embedding(tokens['input_ids'].to(device))

import torch
import torch.nn.functional as F

def find_closest_embeddings(x, embedding_matrix, top_k=1):
    # Normalize both the input tensor x and the embedding matrix
    x = F.normalize(x, dim=1)  # Normalize input tensor along feature dimension
    embedding_matrix = F.normalize(embedding_matrix, dim=1)  # Normalize embedding matrix
    
    # Compute cosine similarity between x and embedding matrix
    similarity = torch.matmul(x, embedding_matrix.T)  # Shape: [10, 50265]
    
    # Find top-k closest embeddings for each tensor in x
    top_k_values, top_k_indices = torch.topk(similarity, top_k, dim=1)
    
    return top_k_indices, top_k_values


# print(output.squeeze(0).shape)
top_k_indices, top_k_values = find_closest_embeddings(output.squeeze(0), text_embedding.weight)
# top_k_indices.shape
indices = tokenizer.decode(top_k_indices.squeeze(-1))
print(indices)

torch.Size([100, 768])
<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><bos>👻🔥😂😁👍🤦‍♀️🤦‍♂️🤷‍♀️🤷‍♂️✌🤞😉😎🎶😢💖🎉🌹💋👏🐱‍💻🐱‍🐉🐱‍👓✔👀😃✨😆🤔🤢🎁🫢 ha ha


In [171]:
words = "👻🔥😂😁👍🤦‍♀️🤦‍♂️🤷‍♀️🤷‍♂️✌🤞😉😎🎶😢💖🎉🌹💋👏🐱‍💻🐱‍🐉🐱‍👓✔👀😃✨😆🤔🤢🎁🫢 ha ha"
tokens = tokenizer(words, truncation=True, padding= 'max_length', max_length=100, return_tensors="pt")
tokens

{'input_ids': tensor([[     0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      2, 242538, 237638, 236471, 238429, 237019, 240525,  67292,
         240525,  68399, 239921,  67292, 239921,  68399, 239529, 241807, 238309,
         238859, 240438, 240116, 239208, 239548, 240315, 240887, 238499, 242993,
         235879, 242482, 242993, 235879, 245092, 242993, 235879, 246943, 237488,
         239220, 239938, 236309, 239312, 238918, 241769, 241227, 248165,    661,
            661]]), 'attention_mask': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_text = "Give me three best book."
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
input_ids

# outputs = gemma.generate(**input_ids, max_new_tokens=200)
# print(tokenizer.decode(outputs[0]))

C:\Users\USER\PycharmProjects\Python3.11.9\pythonProject\.venv\Lib\site-packages\transformers\generation\utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cuda, whereas the model is on cpu. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cpu') before running `.generate()`.
  warnings.warn(


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [127]:
gemma

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((2048,), eps=1e-

# 4. Generator

In [5]:
gemma = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b", device_map="auto",  torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b")
gemma

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2SdpaAttention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), 

In [6]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        # transformer
        self.self_att = nn.MultiheadAttention(768, 1)
        self.multi_att = nn.MultiheadAttention(768, 8)
        self.layer_norm = nn.LayerNorm(768)
        self.linear = nn.Linear(768, 768)
        # gemma
        self.linear_bf_gemma = nn.Linear(768, 50265)
        self.softmax = nn.Softmax(dim=2)
        self.gemma = nn.Sequential(*list(gemma.children())[:-1])
        self.linear_af_gemma = nn.Linear(256, 64)
        self.lm_head = nn.Sequential(*list(gemma.children())[1:])
        self.linearFunnyScore1 = nn.Linear(768, 1)
        self.linearFunnyScore2 = nn.Linear(64, 1)
        
    
    def selfAttention(self, x):
        self_out,_ = self.self_att(x, x, x)
        self_out = self.layer_norm(self_out + x)
        return self_out
    
    def multiheadAttention(self, x):
        multi_out,_ = self.multi_att(x, x, x)
        multi_out = self.linear(multi_out)
        multi_out = self.layer_norm(multi_out + x)
        return multi_out
    
    def coAttention(self, x, y):
        # x: self, y: another
        co_out,_ = self.multi_att(x, y, y)
        co_out = self.linear(co_out)
        co_out = self.layer_norm(co_out + y)
        return co_out
    
    def feedForward(self, x):
        ff_out = self.linear(x)
        ff_out = self.layer_norm(ff_out + x)
        return ff_out
    
    def transformer(self, text, image):
        # self attention module
        image = self.selfAttention(image)
        image = self.feedForward(image)
        # multihead attention module
        text = self.multiheadAttention(text)
        # co-attention module
        visual_attending_textual = self.coAttention(image, text)
        textual_attending_visual = self.coAttention(text, image)
        
        return visual_attending_textual, textual_attending_visual
        
    def gemmaGenerate(self, x):
        with torch.no_grad():
            x = self.linear_bf_gemma(x)
            x = self.softmax(x)
            # get max value of each row, total 32*64
            top_k_values, top_k_indices = torch.topk(x, 1, dim=2, largest=True)
            toGemma = textExtractReverse(top_k_indices).to(device)
            
            # 使用gemma作為model的一部分
            output = self.gemma(toGemma)
            # output[0] = last_hidden_state
            # output[1] = past_key_values
            
        return output[0]
               
    
    def forward(self, text, image):
        max_seq_len = max(text.shape[1], image.shape[1])
        text = nn.functional.pad(text, (0, 0, 0, max_seq_len - text.shape[1]))
        image = nn.functional.pad(image, (0, 0, 0, max_seq_len - image.shape[1]))
        text = text.transpose(0, 1)
        image = image.transpose(0, 1)
        
        visual_attending_textual, textual_attending_visual = self.transformer(text, image)
        
        feature_fusion = visual_attending_textual + textual_attending_visual
        feature_fusion = self.feedForward(feature_fusion)        
        feature_fusion = feature_fusion.squeeze(-1)
        feature_fusion = feature_fusion.transpose(0, 1)
        # gemma generate
        last_hidden_state = self.gemmaGenerate(feature_fusion)
        # linear_af_gemma = nn.Linear(last_hidden_state.shape[-2], 64)
        output_text = last_hidden_state.transpose(1, 2).to(torch.float32)
        output_text = self.linear_af_gemma(output_text).transpose(1, 2).to(torch.bfloat16)
        output_text = self.lm_head(output_text)
        # funny score
        output_funny_score = self.linearFunnyScore1(feature_fusion).squeeze(-1)
        output_funny_score = self.linearFunnyScore2(output_funny_score)

        return output_text, output_funny_score
    
    def generate(self, image, tokenizer, max_length=100):
        result_caption = []
        text = torch.zeros_like(image)
        
        # 有時後空格會失效，所以手動插入空格
        def insert_zeros(list):
            # <pad> = 0, <eos> = 1, <bos> = 2, <unk> = 3, <mask> = 4, <2mass> = 5, [@BOS@] = 6
            zeros = [0] * (2 * len(list) - 1)
            zeros[::2] = list
            return zeros
        
        with torch.no_grad():
            for _ in range(max_length):
                text = text.transpose(0, 1)
                image = image.transpose(0, 1)
                visual_attending_textual, textual_attending_visual = self.transformer(text, image)
                feature_fusion = visual_attending_textual + textual_attending_visual
                feature_fusion = self.feedForward(feature_fusion)        
                feature_fusion = feature_fusion.squeeze(-1)
                feature_fusion = feature_fusion.transpose(0, 1)
                last_hidden_state = self.gemmaGenerate(feature_fusion)
                output_text = self.lm_head(last_hidden_state)

                predicted = output_text.argmax(1)
                result_caption.append(predicted.item()) # not sure
                
                text = insert_zeros(result_caption)
                text = tokenizer.decode(text, skip_special_tokens=False)
                text = text.replace("<pad>", " ").replace("  ", " ").split()
                text = [word for word in text if word[0] != "<"]
                text = " ".join(text)
                
                if predicted.item() == 1: #<eos> = 1 
                    break
                else:
                    text = textExtraction([text])
                    max_seq_len = max(text.shape[1], image.shape[1])
                    text = nn.functional.pad(text, (0, 0, 0, max_seq_len - text.shape[1]))
                    image = nn.functional.pad(image, (0, 0, 0, max_seq_len - image.shape[1]))
            
        return text

# 5. Discriminator

In [76]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.linearFake = nn.Linear(256000, 768)
        self.con_mlp1 = nn.Linear(768, 1)
        self.con_mlp2 = nn.Linear(128, 1)
        self.con_mlp_cd = nn.Linear(256, 1)
        self.unc_mlp1 = nn.Linear(768, 1)
        self.unc_mlp2 = nn.Linear(64, 1)
        
    def forward(self, real_text, fake_text, image):   
        # real_text = [batch_size, 64, 768]
        # fake_text = [batch_size, 64, 256000]
        # image = [batch_size, 64, 768]
        fake_text = self.linearFake(fake_text)
        mismatched_text = torch.roll(real_text, 1, 0)
        #### conditional (contrastive) ####
        C_r = torch.cat((real_text, image), dim=1)
        C_g = torch.cat((fake_text, image), dim=1)
        C_m = torch.cat((mismatched_text, image), dim=1)
        # contrastive discriminator
        C_r = torch.cat((C_r, C_g), dim=1)
        # con_mlp1 
        C_r = self.con_mlp1(C_r).squeeze(-1)
        C_g = self.con_mlp1(C_g).squeeze(-1)
        C_m = self.con_mlp1(C_m).squeeze(-1)
        # con_mlp2/ con_mlp_cd
        C_r = self.con_mlp_cd(C_r).squeeze(-1).unsqueeze(0)
        C_g = self.con_mlp2(C_g).squeeze(-1).unsqueeze(0)
        C_m = self.con_mlp2(C_m).squeeze(-1).unsqueeze(0)
        # con_output
        con_output = torch.cat((C_r, C_g, C_m), dim=0)
        
        #### unconditional ####
        # unc_mlp1
        print("Before MLP:", real_text.shape, fake_text.shape, mismatched_text.shape)
        # Before MLP: torch.Size([32, 64, 768]) torch.Size([32, 64, 768]) torch.Size([32, 64, 768])
        UC_r = self.unc_mlp1(real_text).squeeze(-1)
        UC_g = self.unc_mlp1(fake_text).squeeze(-1)
        UC_m = self.unc_mlp1(mismatched_text).squeeze(-1)

        ##### 拆解問題用 #####
        print("Before MLP:", UC_r.shape, UC_g.shape, UC_m.shape)
        # Before MLP: torch.Size([32, 64]) torch.Size([32, 64]) torch.Size([32, 64])
        UC_r_2 = self.unc_mlp2(UC_r.clone())
        UC_g_2 = self.unc_mlp2(UC_g.clone())
        UC_m_2 = self.unc_mlp2(UC_m.clone()) #### <----------------- Error
        
        print("After MLP:", UC_r_2.shape, UC_g_2.shape, UC_m_2.shape)
        # After MLP: torch.Size([32, 1]) torch.Size([32, 1]) torch.Size([32, 1])
        UC_m_3 = UC_m_2.clone().squeeze(-1).unsqueeze(0)
        UC_g_3 = UC_g_2.clone().squeeze(-1).unsqueeze(0)
        UC_r_3 = UC_r_2.clone().squeeze(-1).unsqueeze(0)
        unc_output = torch.cat((UC_r_3, UC_g_3, UC_m_3), dim=0)
        print("After squeeze and unsqueeze:", UC_r_3.shape, UC_g_3.shape, UC_m_3.shape)
        # After squeeze and unsqueeze: torch.Size([1, 32, 1]) torch.Size([1, 32, 1]) torch.Size([1, 32, 1])
        ##### 原始 Code #####
        # UC_r = self.unc_mlp2(UC_r.clone()).squeeze(-1).unsqueeze(0)
        # UC_g = self.unc_mlp2(UC_g.clone()).squeeze(-1).unsqueeze(0)
        # UC_m = self.unc_mlp2(UC_m.clone()).squeeze(-1).unsqueeze(0)
        # unc_output
        # unc_output = torch.cat((UC_r, UC_g, UC_m), dim=0)
        print(unc_output.shape)
        # torch.Size([3, 32, 1])
        return con_output, unc_output

In [77]:
# empty cuda memory
import gc
torch.cuda.empty_cache()
gc.collect()

561

In [78]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

NetG = Generator().to(device)
NetD = Discriminator().to(device)
optimizer_G = optim.Adam(NetG.parameters(), lr=0.001)
optimizer_D = optim.Adam(NetD.parameters(), lr=0.001)

def generatorLoss(uncondition_logits, condition_logits):
    m = nn.Sigmoid()
    result_fake = (torch.zeros(batch_size)).to(device)
    unc_loss = BCELoss()(m(uncondition_logits[1]), result_fake)
    con_loss = BCELoss()(m(condition_logits[1]), result_fake)
    loss = con_loss + unc_loss
    return loss

def discriminatorLoss(uncondition_logits, condition_logits):
    m = nn.Sigmoid()
    result_true = (torch.ones(batch_size)).to(device)
    result_fake = (torch.zeros(batch_size)).to(device)
    unc_r = BCELoss()(m(condition_logits[0]), result_true)
    unc_f = BCELoss()(m(condition_logits[1]), result_fake)
    unc_m = BCELoss()(m(condition_logits[2]), result_fake)
    con_r = CrossEntropyLoss()(uncondition_logits[0], result_true)
    con_f = CrossEntropyLoss()(uncondition_logits[1], result_fake)
    con_m = CrossEntropyLoss()(uncondition_logits[2], result_fake)
    loss = unc_r + ((unc_f + unc_m)/2) + con_r + ((con_f + con_m)/2)
    return loss

train_losses_G = []
train_losses_D = []
test_losses_G = []
test_losses_D = []

In [79]:
epochs = 10
torch.autograd.set_detect_anomaly(True)
for epoch in range(epochs):
    train_loss_G = 0
    train_loss_D = 0
    test_loss_G = 0
    test_loss_D = 0
    
    ###################################### Train ######################################
    with tqdm(train_loader, unit="batch") as tepoch:
        for text, image, funny_score in tepoch:
            ######################################################
                # (1) Generate fake caption
            ######################################################
            logits, output_funny_score = NetG(text.to(device).to(torch.float32), image.to(device).to(torch.float32))
            con_logits, unc_logits = NetD(text.to(device).to(torch.float32), logits.detach().to(torch.float32), image.to(device).to(torch.float32))
            ######################################################
                # (2) Update Discriminator network
            #####################################################
            optimizer_D.zero_grad()
            loss_D = discriminatorLoss(unc_logits, con_logits)
            loss_D.backward(retain_graph=True)
            optimizer_D.step()
            train_loss_D += loss_D.item()
            ######################################################
                # (3) Update Generator network
            ######################################################
            optimizer_G.zero_grad()
            loss_G = generatorLoss(unc_logits, con_logits)
            loss_G.backward()
            optimizer_G.step()
            train_loss_G += loss_G.item()
            ######################################################
            tepoch.set_postfix('G_loss: ', train_loss_G, ' , D_loss: ', train_loss_D)
    train_losses_G.append(train_loss_G)
    train_losses_D.append(train_loss_D)
    ###################################### Train ######################################
    
    
    ######################################  Test ######################################
    with tqdm(test_loader, unit="batch") as tepoch:
        for text, image, funny_score in tepoch:
            # Generator
            logits, output_funny_score = NetG(text.to(device).to(torch.float32), image.to(device).to(torch.float32))
            # Discriminator
            con_logits, unc_logits = NetD(text.to(device).to(torch.float32), logits.to(torch.float32), image.to(device).to(torch.float32))
            # loss
            loss_G = generatorLoss(unc_logits, con_logits)
            loss_D = discriminatorLoss(unc_logits, con_logits)
            test_loss_G += loss_G.item()
            test_loss_D += loss_D.item()
            tepoch.set_postfix('G_loss: ', test_loss_G, ' , D_loss: ', test_loss_D)
    test_losses_G.append(test_loss_G)
    test_losses_D.append(test_loss_D)
    ######################################  Test ######################################


  0%|          | 0/10 [00:00<?, ?batch/s]C:\Users\USER\PycharmProjects\Python3.11.9\pythonProject\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Before MLP: torch.Size([32, 64, 768]) torch.Size([32, 64, 768]) torch.Size([32, 64, 768])
Before MLP: torch.Size([32, 1, 64]) torch.Size([32, 1, 64]) torch.Size([32, 1, 64])
After MLP: torch.Size([32, 1, 1]) torch.Size([32, 1, 1]) torch.Size([32, 1, 1])
After squeeze and unsqueeze: torch.Size([1, 32]) torch.Size([1, 32]) torch.Size([1, 32])
torch.Size([3, 32])


C:\Users\USER\PycharmProjects\Python3.11.9\pythonProject\.venv\Lib\site-packages\torch\autograd\graph.py:769: UserWarning: Error detected in AddmmBackward0. Traceback of forward call that caused the error:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\USER\PycharmProjects\Python3.11.9\pythonProject\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\USER\PycharmProjects\Python3.11.9\pythonProject\.venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\USER\PycharmProjects\Python3.11.9\pythonProject\.venv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "C:\Users\USER\PycharmProjects\Python3.11.9\pythonProject\.venv\Lib\site-packages\tornado\platform\asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\USER\Ap

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.cuda.FloatTensor [64, 1]], which is output 0 of AsStridedBackward0, is at version 2; expected version 1 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!

# Generate Test

In [302]:
A = [0,155,2,355,4,544,6,788,8,944,10]
def insert_zeros(list):
    # <pad> = 0, <eos> = 1, <bos> = 2, <unk> = 3, <mask> = 4, <2mass> = 5, [@BOS@] = 6
    zeros = [0] * (2 * len(list) - 1)
    zeros[::2] = list
    return zeros

A = insert_zeros(A) 
print(A)
text = tokenizer.decode(A, skip_special_tokens=False)
text = text.replace("<pad>", " ").replace("  ", " ").split()
text = [word for word in text if word[0] != "<"]
text = " ".join(text)
text = textExtraction([text])
print(text.shape)


[0, 0, 155, 0, 2, 0, 355, 0, 4, 0, 544, 0, 6, 0, 788, 0, 8, 0, 944, 0, 10]


100%|██████████| 1/1 [00:00<00:00, 1000.07it/s]

torch.Size([1, 373, 768])
